# **Initialisation**
Initialisation de la base de données spark

In [40]:
import os
import findspark

findspark.init()

import spark


In [41]:
import configparser

config = configparser.ConfigParser()

In [42]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.master("local").appName("Import").getOrCreate()

# **Extraction**
Premier regard sur la base de données 

Extraction des données pertinents via select(). 

In [43]:
df = spark.read.option("multiline","true").json("/Users/nicolassigal/Desktop/Scolaire/ENSTA/3A/ASI322/CVE_Analysis/src/nvdcve-1.1-2022.json")
df = df.select(explode(col("CVE_Items")))
df = df.select("col.publishedDate","col.cve.CVE_data_meta.*","col.cve.description.description_data.value","col.impact.baseMetricV3.cvssV3.*").drop("version").drop("vectorString")

json_files = os.listdir("/Users/nicolassigal/Desktop/Scolaire/ENSTA/3A/ASI322/CVE_Analysis/src/")
for json in json_files:
      if json_files != 'nvdcve-1.1-2022.json' :
            print("Importing", json,"...")
            df_temp = spark.read.option("multiline","true").json("/Users/nicolassigal/Desktop/Scolaire/ENSTA/3A/ASI322/CVE_Analysis/src/"+json)
            df_temp = df_temp.select(explode(col("CVE_Items")))
            df_temp = df_temp.select("col.publishedDate","col.cve.CVE_data_meta.*","col.cve.description.description_data.value","col.impact.baseMetricV3.cvssV3.*").drop("version").drop("vectorString")
            df = df.union(df_temp)

df2 = df.withColumnRenamed("publishedDate","date")
df2 = df2.withColumn("date",df2.date[0:7])
df3 = df2.groupBy("date").count()
df3 = df3.orderBy("date")

print("Columns : ",df2.columns)
df.printSchema()

Importing nvdcve-1.1-2021.json ...
Importing nvdcve-1.1-2017.json ...
Importing nvdcve-1.1-2016.json ...
Importing nvdcve-1.1-2020.json ...
Importing nvdcve-1.1-2019.json ...
Importing nvdcve-1.1-2018.json ...
Importing nvdcve-1.1-2022.json ...
Columns :  ['date', 'ASSIGNER', 'ID', 'value', 'attackComplexity', 'attackVector', 'availabilityImpact', 'baseScore', 'baseSeverity', 'confidentialityImpact', 'integrityImpact', 'privilegesRequired', 'scope', 'userInteraction']
root
 |-- publishedDate: string (nullable = true)
 |-- ASSIGNER: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- value: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- attackComplexity: string (nullable = true)
 |-- attackVector: string (nullable = true)
 |-- availabilityImpact: string (nullable = true)
 |-- baseScore: double (nullable = true)
 |-- baseSeverity: string (nullable = true)
 |-- confidentialityImpact: string (nullable = true)
 |-- integrityImpact: string (nullable 

Affichage des premières lignes et du nombre de samples à disposition pour se faire une idée.

In [44]:
print("Number of available rows : ", df2.count())
df2.show(5)

Number of available rows :  136735
+-------+--------------------+-------------+--------------------+----------------+------------+------------------+---------+------------+---------------------+---------------+------------------+---------+---------------+
|   date|            ASSIGNER|           ID|               value|attackComplexity|attackVector|availabilityImpact|baseScore|baseSeverity|confidentialityImpact|integrityImpact|privilegesRequired|    scope|userInteraction|
+-------+--------------------+-------------+--------------------+----------------+------------+------------------+---------+------------+---------------------+---------------+------------------+---------+---------------+
|2022-03|    secure@intel.com|CVE-2022-0001|[Non-transparent ...|             LOW|       LOCAL|              NONE|      6.5|      MEDIUM|                 HIGH|           NONE|               LOW|  CHANGED|           NONE|
|2022-03|    secure@intel.com|CVE-2022-0002|[Non-transparent ...|             LOW

Essai de la fonction tail pour récupérer des données. Pas idéal. La fonction "collect()" semble bien plus adaptée.

In [45]:
print(df2.tail(1))

[Row(date='2022-11', ASSIGNER='cve@mitre.org', ID='CVE-2022-45939', value=['GNU Emacs through 28.2 allows attackers to execute commands via shell metacharacters in the name of a source-code file, because lib-src/etags.c uses the system C library function in its implementation of the ctags program. For example, a victim may use the "ctags *" command (suggested in the ctags documentation) in a situation where the current working directory has contents that depend on untrusted input.'], attackComplexity=None, attackVector=None, availabilityImpact=None, baseScore=None, baseSeverity=None, confidentialityImpact=None, integrityImpact=None, privilegesRequired=None, scope=None, userInteraction=None)]


# Représentation graphique
Tentative de plot.

In [46]:
import pandas as pd
pandasDF2 = df2.toPandas()
pandasDF3 = df3.toPandas()
print(pandasDF2.columns)

print(pandasDF2.shape[0])

pandasDF2 = pandasDF2[pandasDF2['baseScore'].notna()]

print(pandasDF2.shape[0])


pandasDF2.head(3)

Index(['date', 'ASSIGNER', 'ID', 'value', 'attackComplexity', 'attackVector',
       'availabilityImpact', 'baseScore', 'baseSeverity',
       'confidentialityImpact', 'integrityImpact', 'privilegesRequired',
       'scope', 'userInteraction'],
      dtype='object')
136735
127932


,date,ASSIGNER,ID,value,attackComplexity,attackVector,availabilityImpact,baseScore,baseSeverity,confidentialityImpact,integrityImpact,privilegesRequired,scope,userInteraction
0,2022-03,secure@intel.com,CVE-2022-0001,[Non-transparent sharing of branch predictor s...,LOW,LOCAL,NONE,6.5,MEDIUM,HIGH,NONE,LOW,CHANGED,NONE
1,2022-03,secure@intel.com,CVE-2022-0002,[Non-transparent sharing of branch predictor w...,LOW,LOCAL,NONE,6.5,MEDIUM,HIGH,NONE,LOW,CHANGED,NONE
2,2022-05,secure@intel.com,CVE-2022-0004,[Hardware debug modes and processor INIT setti...,LOW,PHYSICAL,HIGH,6.8,MEDIUM,HIGH,HIGH,NONE,UNCHANGED,NONE


In [47]:
target_column = "baseScore"
target = pandasDF2[target_column]

categorical_columns = [ "attackComplexity",
                        "attackVector",
                        "availabilityImpact",
                        "confidentialityImpact",
                        "integrityImpact",
                        "privilegesRequired",
                        "scope",
                        "userInteraction"]

data = pandasDF2[categorical_columns]

def test(list):
      return list[0]

test_column = pandasDF2["value"].apply(test)

Affichage des types des différentes colonnes

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

dates = pandasDF3["date"]
count = pandasDF3["count"]
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(dates,count,color='y')

plt.xticks(rotation=45, ha="right")
plt.rc('xtick', labelsize=1) 
plt.rc('ytick', labelsize=5) 
plt.rcParams['figure.dpi'] = 750

#pandasDF.loc[116]